In [30]:
library(ggplot2)
library(ggpubr)
library(patchwork)
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [52]:
df = read.csv('df.txt',sep='\t')

In [42]:
sort(paste0(rep(c('Del','Ins','SNV'),4),'_mutant.frequency_',rep(c(0.01,0.4,0.5,0.6),3)))

[1] "Del_mutant.frequency_0.01" "Del_mutant.frequency_0.4" 
 [3] "Del_mutant.frequency_0.5"  "Del_mutant.frequency_0.6" 
 [5] "Ins_mutant.frequency_0.01" "Ins_mutant.frequency_0.4" 
 [7] "Ins_mutant.frequency_0.5"  "Ins_mutant.frequency_0.6" 
 [9] "SNV_mutant.frequency_0.01" "SNV_mutant.frequency_0.4" 
[11] "SNV_mutant.frequency_0.5"  "SNV_mutant.frequency_0.6"

In [138]:
df_transform_raw <- bind_rows(lapply(sort(paste0(rep(c('Del','Ins','SNV'),4),'_mutant.frequency_',rep(c(0.01,0.4,0.5,0.6),3))),function(x){
    tmp_df <- df[,c('Type.of.chemical.modification',x)]
    tmp_df$Value <- tmp_df[,x]
    tmp_df$type <- strsplit(x,'_')[[1]][1]
    tmp_df$AF <- strsplit(x,'_')[[1]][3]
    return(tmp_df[,c('Type.of.chemical.modification','Value','AF','type')])}))
df_transform_raw[,'Group'] <- c('E1Ctrl'='Group1','91G'='Group1','92A'='Group1','93C'='Group1','94T'='Group1',
                      'E2Ctrl'='Group2','EtO'='Group2','S'='Group2',
                      'E2Ctrl'='Group3','LNA'='Group3','LNATC'='Group3')[df_transform_raw[,'Type.of.chemical.modification']]

In [1]:
df_transform <- aggregate(list('Value'=df_transform_raw$Value),
                          by=list('Type.of.chemical.modification'=df_transform_raw$Type.of.chemical.modification,
                                  'AF'=df_transform_raw$AF,
                                  'type'=df_transform_raw$type,
                                  'Group'=df_transform_raw$Group),FUN = function(x) {c('mean' = mean(x), 'sd' = sd(x))})
df_transform$mean <- df_transform$Value[,'mean']
df_transform$sd <- df_transform$Value[,'sd']
df_transform$Value <- NULL

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (2170006327.py, line 1)

In [2]:
plot_AF_Group_box <- function(Sample,Group,AF){
    tmp_df <- df_transform[df_transform$Type.of.chemical.modification%in%Sample&df_transform$AF==AF,]
    tmp_df_raw <- df_transform_raw[df_transform_raw$Type.of.chemical.modification%in%Sample&df_transform$AF==AF,]
    my_comparisons <- combn(unique(tmp_df$Type.of.chemical.modification), 2)
    my_comparisons <- lapply(1:ncol(my_comparisons),function(x){return(my_comparisons[,x])})
    
    p1 <- 
    ggplot(tmp_df_raw,aes(x=Type.of.chemical.modification,y=Value))+
    geom_point(aes(x=Type.of.chemical.modification,y=Value),color='black')+
    geom_bar(data=tmp_df,aes(x=Type.of.chemical.modification,y=mean,fill=Type.of.chemical.modification),color='black',lwd=0.5,width=0.8,stat='identity')+
    geom_errorbar(data=tmp_df,aes(x=Type.of.chemical.modification,y=mean,ymin = mean-sd, ymax = mean+sd), width = 0.2,color='black',lwd=0.5)+ 
    scale_fill_manual(values = c('#E64B35FF','#4DBBD5FF','#00A087FF','#3C5488FF',
                                 '#F39B7FFF','#8491B4FF',
                                 '#91D1C2FF','#DC0000FF','#7E6148FF','#B09C85FF'))+
    stat_compare_means(data=tmp_df_raw,mapping=aes(x=Type.of.chemical.modification,y=Value),comparisons = my_comparisons,label ="p.signif",size=6)+
    facet_wrap(~type,scale='free')+
    theme_classic()+
    theme(text = element_text(size=18))+
    ggtitle(paste0('AF=',AF,', ',Group))+
    ylab('Frequency')
    p1
}

SyntaxError: invalid syntax (4034039305.py, line 1)

In [3]:
options(repr.plot.width=15,repr.plot.height=16)
plot_AF_Group_box(Sample=c('E1Ctrl','91G','92A','93C','94T'),Group='Group1',AF='0.6')+
plot_AF_Group_box(Sample=c('E2Ctrl','EtO','S'),Group='Group2',AF='0.6')+
plot_AF_Group_box(Sample=c('E2Ctrl','LNA','LNATC'),Group='Group3',AF='0.6')+plot_layout(nrow=3)

SyntaxError: expression cannot contain assignment, perhaps you meant "=="? (3632685823.py, line 1)

In [234]:
for(i in c('0.01','0.4','0.5','0.6')){
    p <- plot_AF_Group_box(Sample=c('E1Ctrl','91G','93C','94T'),Group='Group1',AF=i)+
         plot_AF_Group_box(Sample=c('E2Ctrl','EtO','S'),Group='Group2',AF=i)+
         plot_AF_Group_box(Sample=c('E2Ctrl','LNA','LNATC'),Group='Group3',AF=i)+plot_layout(nrow=3)
    plot_layout(nrow=3)
    pdf(paste0('R_Plot/','AF_',i,'.pdf'),width=15,height=16)
    print(p)
    dev.off()
}

Warning message in wilcox.test.default(c(0.000141064, 0.000384287, 9.4043e-05, 5.48982e-05:
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(c(0.000591475, 0.000315483, 0.000590474, :
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(c(0.000112902, 0.000272285, 5.74776e-05, :
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(c(0.000154016, 0.001148031, 0.000112503, :
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(c(0.000154016, 0.001148031, 0.000112503, :
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(c(0.000154016, 0.001148031, 0.000112503, :
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(c(0, 0.000136203, 0, 2.02294e-05, 3.94353e-05, :
“cannot compute exact p-value with ties”
Warning message in wilcox.test.default(c(0, 0.000136203, 0, 2.02294e-05, 3.94353e-05, :
“cannot compute exact p-value with 